<a href="https://colab.research.google.com/github/JihunSKKU/PyMongo/blob/main/MongoDB_Aggregation_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.5 MB/s eta 0:00:00


In [3]:
import pymongo
from pymongo import MongoClient

In [27]:
CONNECTION_STRING = "---"
my_client = MongoClient(CONNECTION_STRING)

database = my_client['root']
collection = database['products2']

In [28]:
import json

doc_list = []
for line in open('products.json', 'r'):
    doc_list.append(json.loads(line))

In [29]:
try:
    collection.insert_many(doc_list)
except pymongo.errors.BulkWriteError:
    pass
    # if there is a document with the same id as the added document
    # list, do not include the list itself.

## Aggregation Operation

### $match

In [30]:
samsungs = collection.aggregate([{"$match": {"brand": "Samsung"}}])
for samsung in samsungs:
    print(samsung)

{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}


### $project

0: Specifies the inclusion of a field

In [32]:
brands = collection.aggregate([{"$project": {"brand": 0}}])
for brand in brands:
    print(brand)

{'_id': 'dial1', 'name': 'iPhone 14', 'type': 'phone', 'price': 250, 'warranty_years': 1, 'available': True}
{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial3', 'name': 'Galaxy S23', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 0}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 0.25, 

1: Specifies the exclusion of a field

In [31]:
brands = collection.aggregate([{"$project": {"brand": 1}}])
for brand in brands:
    print(brand)

{'_id': 'dial1', 'brand': 'Apple'}
{'_id': 'dial2', 'brand': 'Apple'}
{'_id': 'dial3', 'brand': 'Samsung'}
{'_id': 'dial4', 'brand': 'Samsung'}
{'_id': 'dial5'}
{'_id': 'dial6'}
{'_id': 'dial7'}
{'_id': 'dial8'}
{'_id': 'dial9'}
{'_id': 'dial10'}
{'_id': 'dial11'}


### $group

In [33]:
brands = collection.aggregate([
    {
        "$group": {
            "_id": "$brand",
            "names": {"$push": "$name"}
        }
    }
])
for brand in brands:
    print(brand)

{'_id': None, 'names': ['iPhone Charger', 'Galaxy Charger', 'iPhone 14 Case Green', 'iPhone 14 Case Black', 'iPhone 14 Mini Case Yellow', 'Galaxy S23 Case Green', 'Galaxy S23 Ultra Case Red']}
{'_id': 'Samsung', 'names': ['Galaxy S23', 'Galaxy S23 Ultra']}
{'_id': 'Apple', 'names': ['iPhone 14', 'iPhone 14 Pro']}


### $sort

- Ascending: 1
- Descending: -1

In [34]:
name_sorts = collection.aggregate([{"$sort": {"name": 1}}])
for name_sort in name_sorts:
    print(name_sort)

{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial10', 'name': 'Galaxy S23 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 10, 'warranty_years': 0}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 0.25, 'available': True, 'for': 'dial4'}
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 250, 'warranty_years': 1, 'available': True}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 0.25, 'available': False, '

### $sum

In [36]:
price_sums = collection.aggregate([{"$group": {"_id": "$type", "price": {"$sum": "$price"}}}])
for price_sum in price_sums:
    print(price_sum)

{'_id': ['accessory', 'case'], 'price': 59}
{'_id': 'phone', 'price': 1030}
{'_id': ['accessory', 'charger'], 'price': 40}


### $unwind


In [38]:
unwind_types = collection.aggregate([
    {'$project': {'type': 1}},
    {'$unwind': '$type'}
])
for unwind_type in unwind_types:
    print(unwind_type)

{'_id': 'dial1', 'type': 'phone'}
{'_id': 'dial2', 'type': 'phone'}
{'_id': 'dial3', 'type': 'phone'}
{'_id': 'dial4', 'type': 'phone'}
{'_id': 'dial5', 'type': 'accessory'}
{'_id': 'dial5', 'type': 'charger'}
{'_id': 'dial6', 'type': 'accessory'}
{'_id': 'dial6', 'type': 'charger'}
{'_id': 'dial7', 'type': 'accessory'}
{'_id': 'dial7', 'type': 'case'}
{'_id': 'dial8', 'type': 'accessory'}
{'_id': 'dial8', 'type': 'case'}
{'_id': 'dial9', 'type': 'accessory'}
{'_id': 'dial9', 'type': 'case'}
{'_id': 'dial10', 'type': 'accessory'}
{'_id': 'dial10', 'type': 'case'}
{'_id': 'dial11', 'type': 'accessory'}
{'_id': 'dial11', 'type': 'case'}


### Exercise

In [39]:
chargers = collection.aggregate([{"$match": {"type": "charger"}}])
for charger in chargers:
    print(charger)

{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}


In [40]:
price_sorts = collection.aggregate([{"$sort": {"price": -1}}])
for price_sort in price_sorts:
    print(price_sort)

{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 250, 'warranty_years': 1, 'available': True}
{'_id': 'dial3', 'name': 'Galaxy S23', 'brand': 'Samsung', 'type': 'phone', 'price': 200, 'warranty_years': 1, 'available': False}
{'_id': 'dial5', 'name': 'iPhone Charger', 'type': ['accessory', 'charger'], 'price': 25, 'warranty_years': 0.25, 'for': ['dial1', 'dial2']}
{'_id': 'dial6', 'name': 'Galaxy Charger', 'type': ['accessory', 'charger'], 'price': 15, 'warranty_years': 0.25, 'for': ['dial3', 'dial4']}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 0.25, 'available': True, 'for': 'dial4'}
{'_id': 'dial7', 

In [41]:
price_sums = collection.aggregate([
    {"$match": {"$or": [{"type": "charger"}, {"type": "case"}]}},
    {"$group": {"_id": "$type", "price": {"$sum": "$price"}}}
])
for price_sum in price_sums:
    print(price_sum)

{'_id': ['accessory', 'case'], 'price': 59}
{'_id': ['accessory', 'charger'], 'price': 40}


In [42]:
phone_sorts = collection.aggregate([
    {"$match": {"$and": [
                        {"type": "phone"},
                        {"price": {"$gte": 250}}
                        ]
                }
    },
    {"$sort": {"price": -1}}
])
for phone_sort in phone_sorts:
    print(phone_sort)

{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 250, 'warranty_years': 1, 'available': True}


In [44]:
phone_sorts = collection.aggregate([
    {"$match": {"$and": [
                        {"type": "phone"},
                        {"price": {"$gte": 250}}
                        ]
                }
    },
    {"$sort": {"price": -1}}
])
for phone_sort in phone_sorts:
    print(phone_sort)


{'_id': 'dial2', 'name': 'iPhone 14 Pro', 'brand': 'Apple', 'type': 'phone', 'price': 300, 'warranty_years': 1, 'available': False}
{'_id': 'dial4', 'name': 'Galaxy S23 Ultra', 'brand': 'Samsung', 'type': 'phone', 'price': 280, 'warranty_years': 1, 'available': False}
{'_id': 'dial1', 'name': 'iPhone 14', 'brand': 'Apple', 'type': 'phone', 'price': 250, 'warranty_years': 1, 'available': True}


In [45]:
case_colors = collection.aggregate([
    {"$match": {"type": "case"}},
    {"$group": {"_id": "$color", "name": {"$push": "$name"}}}
])
for case_color in case_colors:
    print(case_color)

{'_id': 'yellow', 'name': ['iPhone 14 Mini Case Yellow']}
{'_id': 'green', 'name': ['iPhone 14 Case Green', 'Galaxy S23 Case Green']}
{'_id': 'black', 'name': ['iPhone 14 Case Black']}
{'_id': 'red', 'name': ['Galaxy S23 Ultra Case Red']}


In [46]:
results = collection.aggregate([
    {'$match': {'type': {'$all': ['accessory', 'case']}}},
])
for doc in results:
    print(doc)

{'_id': 'dial7', 'name': 'iPhone 14 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 12, 'warranty_years': 0}
{'_id': 'dial8', 'name': 'iPhone 14 Case Black', 'type': ['accessory', 'case'], 'color': 'black', 'price': 12, 'warranty_years': 0.25, 'available': False, 'for': ['dial1', 'dial2']}
{'_id': 'dial9', 'name': 'iPhone 14 Mini Case Yellow', 'type': ['accessory', 'case'], 'color': 'yellow', 'price': 10, 'warranty_years': 0}
{'_id': 'dial10', 'name': 'Galaxy S23 Case Green', 'type': ['accessory', 'case'], 'color': 'green', 'price': 10, 'warranty_years': 0}
{'_id': 'dial11', 'name': 'Galaxy S23 Ultra Case Red', 'type': ['accessory', 'case'], 'color': 'red', 'price': 15, 'warranty_years': 0.25, 'available': True, 'for': 'dial4'}


## Word Counting with Aggregation Pipeline

In [48]:
text_collection = database["wordcount"]
text_collection.insert_many([
    {"text":"hello world"},
    {"text":"hello hello world"},
    {"text":"hello hello hello word"}
])

InsertManyResult([ObjectId('673348419fd9780179670e08'), ObjectId('673348419fd9780179670e09'), ObjectId('673348419fd9780179670e0a')], acknowledged=True)

In [53]:
for doc in text_collection.find():
    print(doc)

{'_id': ObjectId('6733482a9fd9780179670e05'), 'text': 'hello world'}
{'_id': ObjectId('6733482a9fd9780179670e06'), 'text': 'hello hello world'}
{'_id': ObjectId('6733482a9fd9780179670e07'), 'text': 'hello hello hello word'}


In [54]:
word_counts = text_collection.aggregate([
    {'$project': {'words': {'$split': ['$text', ' ']}}}])
for doc in word_counts:
    print(doc)

{'_id': ObjectId('6733482a9fd9780179670e05'), 'words': ['hello', 'world']}
{'_id': ObjectId('6733482a9fd9780179670e06'), 'words': ['hello', 'hello', 'world']}
{'_id': ObjectId('6733482a9fd9780179670e07'), 'words': ['hello', 'hello', 'hello', 'word']}


In [55]:
word_counts = text_collection.aggregate([
    {'$project': {'words': {'$split': ['$text', ' ']}}},
    {'$unwind': '$words'}])
for doc in word_counts:
    print(doc)

{'_id': ObjectId('6733482a9fd9780179670e05'), 'words': 'hello'}
{'_id': ObjectId('6733482a9fd9780179670e05'), 'words': 'world'}
{'_id': ObjectId('6733482a9fd9780179670e06'), 'words': 'hello'}
{'_id': ObjectId('6733482a9fd9780179670e06'), 'words': 'hello'}
{'_id': ObjectId('6733482a9fd9780179670e06'), 'words': 'world'}
{'_id': ObjectId('6733482a9fd9780179670e07'), 'words': 'hello'}
{'_id': ObjectId('6733482a9fd9780179670e07'), 'words': 'hello'}
{'_id': ObjectId('6733482a9fd9780179670e07'), 'words': 'hello'}
{'_id': ObjectId('6733482a9fd9780179670e07'), 'words': 'word'}


In [56]:
word_counts = text_collection.aggregate([
    {'$project': {'words': {'$split': ['$text', ' ']}}},
    {'$unwind': '$words'},
    {'$group': {'_id': {'word': '$words'}, 'count': {'$sum': 1}}}
])
for doc in word_counts:
    print(doc)


{'_id': {'word': 'hello'}, 'count': 6}
{'_id': {'word': 'world'}, 'count': 2}
{'_id': {'word': 'word'}, 'count': 1}


### Exercise

In [57]:
new_collection = database["new_wordcount"]
doc_list = []
for line in open('eng_collection.json', 'r'):
    doc_list.append(json.loads(line))
for doc in doc_list:
    try:
        new_collection.insert_one(doc)
    except pymongo.errors.DuplicateKeyError:
        pass

In [58]:
new_wordcount_result = new_collection.aggregate([
    {'$project': {'words': {'$split': ['$text', ' ']}}},
    {'$unwind': '$words'},
    {'$group': {'_id': {'word': '$words'}, 'count': {'$sum': 1}}},
    {'$project': {'value': {'count': '$count'}}},
    {'$sort': {'value': -1}}
])
for doc in new_wordcount_result:
    print(doc)

{'_id': {'word': 'is'}, 'value': {'count': 7}}
{'_id': {'word': 'the'}, 'value': {'count': 6}}
{'_id': {'word': 'not'}, 'value': {'count': 5}}
{'_id': {'word': 'a'}, 'value': {'count': 5}}
{'_id': {'word': 'life'}, 'value': {'count': 4}}
{'_id': {'word': 'to'}, 'value': {'count': 4}}
{'_id': {'word': 'than'}, 'value': {'count': 3}}
{'_id': {'word': 'no'}, 'value': {'count': 3}}
{'_id': {'word': 'for'}, 'value': {'count': 3}}
{'_id': {'word': 'in'}, 'value': {'count': 3}}
{'_id': {'word': 'give'}, 'value': {'count': 3}}
{'_id': {'word': 'day'}, 'value': {'count': 3}}
{'_id': {'word': 'can'}, 'value': {'count': 3}}
{'_id': {'word': 'do'}, 'value': {'count': 3}}
{'_id': {'word': 'it'}, 'value': {'count': 3}}
{'_id': {'word': 'over'}, 'value': {'count': 2}}
{'_id': {'word': 'better'}, 'value': {'count': 2}}
{'_id': {'word': 'we'}, 'value': {'count': 2}}
{'_id': {'word': 'of'}, 'value': {'count': 2}}
{'_id': {'word': 'great'}, 'value': {'count': 2}}
{'_id': {'word': 'laughter'}, 'value': {'